#### Loading packages

In [177]:
import torch
from astropy.io import fits
import glob
import numpy as np
import pandas as pd

In [178]:
if (torch.cuda.is_available()):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

# intel HD does not support CUDA

cpu


In [179]:
# classical approach to compare with ANN
def rolo_b_me(alpha, emission, incidence):
    '''
    Here we obtain the reflectance for any photometric angle (using degrees as input)  
    according to our classic photometric modeling. This model consists of the Sommel-Seeliger 
    disk function and ROLO as a phase function.

    Use example: for phase=30º, emission=0º and incidence=30º - ipwg(30, 0, 30)
    
    Inputs: phase, emission and incidence angles
    Output: reflectance
    '''
    # parameters
    C_0, C_1, A_0, A_1, A_2, A_3, A_4=np.loadtxt('parameters.txt')  
    
    # converting degrees to radians:
    emission=np.deg2rad(emission)    
    incidence=np.deg2rad(incidence)
    
    # computing reflectance according to this photometric model:
    disk = (np.cos(incidence)/(np.cos(incidence)+np.cos(emission)))
    phase = C_0*np.exp(-C_1*alpha) + A_0 + A_1*alpha + A_2*(alpha)**2 + A_3*(alpha)**3  + A_4*(alpha)**4 
    
    reflectance = disk * phase
    
    return reflectance

In [180]:
def data_preparation(ID):
    '''
    Here we extract features and labels using the ID of each image.
    Later we convert these data in the apropiate format for training or validation.
    It calls to data_normalization() function.
    '''
    
    iof_n, phase_n, emission_n, incidence_n = data_normalization(ID)
    
    
    # packing data
    features = np.zeros((len(phase_n),3)) 
    features[:,0] = phase_n
    features[:,1] = emission_n
    features[:,2] = incidence_n
    
    features = torch.FloatTensor(features)
    features = features.to(device)
    
    labels = np.zeros((len(iof_n),1))
    labels[:,0] = iof_n
    
    labels = torch.FloatTensor(labels)
    labels = labels.to(device)
    
    return features, labels

In [181]:
def data_normalization(ID):
    '''
    In this function we rescale data between 0 and 1, a very important step before trainning a neural network.
    It calls to loading_and_cleaning_data() function.
    '''
    iof_, phase_, emission_, incidence_ = loading_and_cleaning_data(ID)
       
    #rescaling data
    iof_n=iof_/0.06; phase_n=phase_/90; emission_n=emission_/82; incidence_n=incidence_/82

    return iof_n, phase_n, emission_n, incidence_n

In [182]:
def loading_and_cleaning_data(ID):
    '''
    This function is for loading iof, phase, emission, incidence values for each image. 
    '''
    #criterion for removing data:
    em, eM = (0, 82)        # emission limits  
    im, iM = (0, 82)          # incidence limits
    pm, pM = (0, 90)       # phase limits
    rm, rM =  (0.001,1)  # reflectance higher limits
    
    fits_file = fits.getdata(ID, ignore_missing_end=True)
    
    iof = fits_file[0]
    phase = fits_file[1]
    emission = fits_file[2]
    incidence = fits_file[3]
    
    idxsort = (emission >= em) & (emission <= eM) & \
              (incidence >= im) & (incidence <= iM) & \
              (phase >= pm) & (phase <= pM) & \
              (iof >= rm) & (iof <= rM) 
    
    return iof[idxsort], phase[idxsort], emission[idxsort], incidence[idxsort] 

In [183]:
# loading ID list of files:
files_ID=[]
for i in sorted(glob.glob('reduced_phocubes/*reduce.fits')):
    files_ID.append(i)

In [184]:
len(files_ID)

951

#### Preparing training and validation data

In [185]:
import random

# defining global tensors 
final_training_features = torch.zeros(0)
final_training_labels = torch.zeros(0)
final_validation_features = torch.zeros(0)
final_validation_labels = torch.zeros(0)

final_training_features = final_training_features.to(device)
final_training_labels = final_training_labels.to(device)
final_validation_features = final_validation_features.to(device)
final_validation_labels = final_validation_labels.to(device)

for file in files_ID:
    
    # loading training data from files_ID list
    _features, _labels = data_preparation(file)
    if len(_features) != 0:
        np.random.seed(10)
        # Selecting 1000 random values of this image
        try:
            mask_temp = np.random.choice(len(_features),1000,replace=False)
        
        except:
            mask_temp = np.random.choice(len(_features),len(_features),replace=False)
        
        split = int(len(mask_temp)*(9/10))
        mask_train = mask_temp[0:split]
        mask_val = mask_temp[split:]
        
        training_features =  _features[mask_train]
        final_training_features = torch.cat([final_training_features,training_features])
        
        training_labels = _labels[mask_train]
        final_training_labels = torch.cat([final_training_labels,training_labels])
        
        validation_features =  _features[mask_val]
        final_validation_features=torch.cat([final_validation_features,validation_features])

        validation_labels = _labels[mask_val]
        final_validation_labels=torch.cat([final_validation_labels,validation_labels])

In [186]:
print(len(final_training_features), len(final_validation_features))

417004 46477


#### Setting up and defining a neural network

In [197]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes, n_output):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_sizes[0])
        torch.nn.init.kaiming_normal_(self.fc1.weight) 
        self.fc2 = torch.nn.Linear(hidden_sizes[0], hidden_sizes[1])
        torch.nn.init.kaiming_normal_(self.fc2.weight)
        self.fc3 = torch.nn.Linear(hidden_sizes[1], hidden_sizes[2])
        torch.nn.init.kaiming_normal_(self.fc3.weight)
        self.fc4 = torch.nn.Linear(hidden_sizes[2], hidden_sizes[3])
        torch.nn.init.kaiming_normal_(self.fc4.weight)
        self.fc5 = torch.nn.Linear(hidden_sizes[3], hidden_sizes[4])
        torch.nn.init.kaiming_normal_(self.fc5.weight)
        self.fc6 = torch.nn.Linear(hidden_sizes[4], n_output)
        torch.nn.init.kaiming_normal_(self.fc6.weight)
        self.relu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid() 
        
  
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.relu(self.fc4(out))
        out = self.relu(self.fc5(out))
        x = self.sigmoid(self.fc6(out))
        return x

In [198]:
input_size = 3
hidden_sizes = [15,15,15,15,15]
n_output = 1
torch.manual_seed(7)
model = Net(input_size, hidden_sizes, n_output).to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)
criterion = torch.nn.MSELoss()  

In [199]:
epochs=200
validation_error=[]
batch_extension = 1000
n_batches = len(final_training_features) // batch_extension 

for epoch in range(epochs):
    model.train()
    for b in range(n_batches):
            left = b*batch_extension
            right = (b+1) * batch_extension  
            optimizer.zero_grad()
            prediction = model(final_training_features[left:right,:]) 
            loss = criterion(prediction, final_training_labels[left:right])
            loss.backward() 
            optimizer.step() 
            
    model.eval()
    with torch.no_grad():
        prediction_val = model(final_validation_features)
    Validation_error = criterion(prediction_val, final_validation_labels)
    validation_error.append(Validation_error.cpu().numpy())  
    
    if (epoch % 10 == 0 and epoch != 0):
    
        print(f'{epoch} - L_tr={loss.item()} L_val={Validation_error.item()}')
np.savetxt('validation_error',validation_error)

10 - L_tr=0.006487206555902958 L_val=0.011827516369521618
20 - L_tr=0.0010473834117874503 L_val=0.005182414315640926
30 - L_tr=0.0006265772390179336 L_val=0.00434835022315383
40 - L_tr=0.0005859573138877749 L_val=0.004151467699557543
50 - L_tr=0.0005723173962906003 L_val=0.004071469884365797
60 - L_tr=0.0005676352884620428 L_val=0.004025597125291824
70 - L_tr=0.0005664670024998486 L_val=0.003994517028331757
80 - L_tr=0.0005668990197591484 L_val=0.003971249796450138
90 - L_tr=0.0005680486792698503 L_val=0.003952764440327883
100 - L_tr=0.0005694955470971763 L_val=0.003937585279345512
110 - L_tr=0.0005710669793188572 L_val=0.003924873191863298
120 - L_tr=0.000572633056435734 L_val=0.003914106637239456
130 - L_tr=0.0005740716005675495 L_val=0.0039049636106938124
140 - L_tr=0.0005752513534389436 L_val=0.003897223388776183
150 - L_tr=0.0005760762142017484 L_val=0.0038907499983906746
160 - L_tr=0.0005765488022007048 L_val=0.003885445650666952
170 - L_tr=0.0005767031107097864 L_val=0.003881243

#### Neural network eficciency using MSE:

In [200]:
model.eval()
with torch.no_grad():
    prediction_NEURAL = model(final_validation_features)
eff_NEURAL=torch.mean((prediction_NEURAL-final_validation_labels)**2)
eff_NEURAL

tensor(0.0039)

#### ROLO eficciency using MSE:

In [201]:
phase=final_validation_features.cpu().numpy()[:,0]*90
emission=final_validation_features.cpu().numpy()[:,1]*82
incidence=final_validation_features.cpu().numpy()[:,2]*82
prediction_ROLO=torch.tensor(rolo_b_me(phase, emission, incidence)/0.06).to(device)

In [202]:
eff_ROLO=torch.mean((prediction_ROLO-final_validation_labels.t())**2)
print(eff_ROLO)

tensor(0.0044)


#### Improvement percentage

In [203]:
print((eff_ROLO - eff_NEURAL)*100/eff_ROLO)

tensor(11.9504)
